In [53]:
from collections import defaultdict, deque
import itertools
import numpy as np
import random
from enum import Enum
from typing import List, Tuple, Generator, Type
from dataclasses import dataclass
import time
import os
import bisect
from statistics import mean
from matplotlib import pyplot as plt


def list_files_in_folder(folder_path):
    items = []
    try:
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)

            # Check if it's a file and not a directory (excluding .ipynb_checkpoints)
            if os.path.isfile(item_path) and '.ipynb_checkpoints' not in item and "GVNS" not in item and "VND" not in item and "GRASP" not in item:
                items.append(item_path)
        return items
    except FileNotFoundError:
        print(f"Folder not found: {folder_path}")
        return []

folder_path = "/content/competition"
test_folder_path_small = "/content/test/small"
test_folder_path_med = "/content/test/med"
test_folder_path_med_large = "/content/test/med_large"
test_folder_path_large = "/content/test/large"


items = list_files_in_folder(folder_path)
items_test_small = list_files_in_folder(test_folder_path_small)
items_test_med = list_files_in_folder(test_folder_path_med)
items_test_med_large = list_files_in_folder(test_folder_path_med_large)
items_test_large = list_files_in_folder(test_folder_path_large)

class Graph:
    def __init__(self, U_size, V_size):
        self.U = list(range(1, U_size + 1))  # Nodes in fixed layer U
        self.V = list(range(U_size + 1, U_size + V_size + 1))  # Nodes in V
        self.edges = []  # List of edges (u, v, weight)
        self.constraints = defaultdict(list)  # Constraints as adjacency list for V
        self.in_degree = {v: 0 for v in self.V}   # Dictionary to store in-degree of nodes in V
        self.node_edges = defaultdict(list)  # Edges connected to each node
        self.solution_costs = {}  # Store costs for each solution

    def add_node_U(self, node):
        self.U.append(node)

    def add_node_V(self, node):
        self.V.append(node)
        self.in_degree[node] = 0  # Initialize in-degree for nodes in V

    def add_edge(self, u, v, weight):
        self.edges.append((u, v, weight))
        self.node_edges[u].append((v, weight))
        self.node_edges[v].append((u, weight))

    def add_constraint(self, v1, v2):
        self.constraints[v1].append(v2)
        self.in_degree[v2] += 1  # Update in-degree due to precedence constraint

def verify_solution(graph, solution: List[int]) -> bool:
        """Verify if a solution is valid."""
        # Check if all nodes are present exactly once
        if set(solution) != set(graph.V):
            return False

        # Check if constraints are satisfied
        positions = {node: idx for idx, node in enumerate(solution)}
        for v1 in graph.V:
            for v2 in graph.constraints[v1]:
                if positions[v1] >= positions[v2]:
                    return False
        return True

# Utility function for timing
class Timer:
    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, *args):
        self.end = time.perf_counter()
        self.delta = self.end - self.start


def load_instance(filename):
    with open(filename, 'r') as file:
        U_size, V_size, C_size, E_size = map(int, file.readline().split())
        graph = Graph(U_size, V_size)

        # Read constraints section
        line = file.readline().strip()
        while line != "#constraints":
            line = file.readline().strip()

        for _ in range(C_size):
            v, v_prime = map(int, file.readline().split())
            graph.add_constraint(v, v_prime)

        # Read edges section
        line = file.readline().strip()
        while line != "#edges":
            line = file.readline().strip()

        for _ in range(E_size):
            u, v, weight = file.readline().split()
            graph.add_edge(int(u), int(v), float(weight))

    return graph

class BinaryIndexedTree:
    def __init__(self, size):
        self.size = size
        self.tree = [0] * (size + 1)

    def update(self, idx, val):
        idx += 1  # Convert to 1-based indexing
        while idx <= self.size:
            self.tree[idx] += val
            idx += idx & (-idx)

    def query(self, idx):
        idx += 1  # Convert to 1-based indexing
        total = 0
        while idx > 0:
            total += self.tree[idx]
            idx -= idx & (-idx)
        return total

def cost_function_bit(graph, permutation):
    """
    Alternative implementation using Binary Indexed Tree for very large graphs.
    Time complexity: O(E log E) where E is number of edges.

    Args:
        graph: Graph object containing edges and weights
        permutation: List representing the ordering of nodes in layer V

    Returns:
        float: Total cost of crossings
    """

    tuple_permutation = tuple(permutation)

    cost = graph.solution_costs.get(tuple_permutation, None)

    if cost is not None:
        return cost

    position = {node: idx for idx, node in enumerate(permutation)}
    max_pos = len(permutation)

    # Sort edges by u value
    edges = [(u, position[v], w) for u, v, w in graph.edges]
    edges.sort()

    total_cost = 0
    bit = BinaryIndexedTree(max_pos)
    weight_sum = BinaryIndexedTree(max_pos)

    # Process edges in order of increasing u
    for i, (u, pos_v, w) in enumerate(edges):
        # Count crossings with previous edges
        crossings = bit.query(max_pos - 1) - bit.query(pos_v)
        total_cost += crossings * w

        # Add contribution from weights of crossed edges
        weight_contribution = (weight_sum.query(max_pos - 1) -
                             weight_sum.query(pos_v))
        total_cost += weight_contribution

        # Update BITs
        bit.update(pos_v, 1)
        weight_sum.update(pos_v, w)

    graph.solution_costs[tuple_permutation] = total_cost

    return total_cost

Folder not found: /content/competition
Folder not found: /content/test/med
Folder not found: /content/test/med_large
Folder not found: /content/test/large


In [64]:
from typing import List, Optional

class DeterministicConstruction:
    """
    A deterministic construction algorithm using a fast heuristic based on
    U-layer positions and edge weights.
    """

    def __init__(self, graph: 'Graph'):
        self.graph = graph
        self.pi: List[int] = []
        self._precompute_node_metrics()

    def _precompute_node_metrics(self) -> None:
        """
        Precompute node metrics for efficient lookup:
        - Total edge weight per node
        - Average U-layer position of connected nodes
        """
        self.node_weights = defaultdict(int)
        self.avg_u_positions = defaultdict(float)
        self.u_connections = defaultdict(list)

        # Compute total weights and collect U positions
        for u, v, weight in self.graph.edges:
            self.node_weights[v] += weight
            self.u_connections[v].append(u)

        # Compute average U position
        for v in self.graph.V:
            if self.u_connections[v]:
                self.avg_u_positions[v] = sum(self.u_connections[v]) / len(self.u_connections[v])
            else:
                self.avg_u_positions[v] = len(self.graph.U) / 2  # Middle position if no connections

    def _get_best_candidate(self, candidates: deque, current_len: int) -> Optional[int]:
        """
        Select best candidate based on a simple scoring function that considers:
        1. Average position in U layer (prefer nodes whose U connections align with current position)
        2. Edge weights (prefer heavier edges)

        Args:
            candidates: Collection of candidate nodes
            current_len: Current length of placement

        Returns:
            Best candidate node or None if candidates is empty
        """
        if not candidates:
            return None

        # Target position in U layer based on current placement
        target_u_pos = (current_len / len(self.graph.V)) * len(self.graph.U)

        # Find node that best matches target position and has appropriate weight
        best_node = min(
            candidates,
            key=lambda v: (
                abs(self.avg_u_positions[v] - target_u_pos) * 0.9 +  # Position alignment
                -self.node_weights[v] * 0.1 # Weight consideration (negative to prefer higher weights)
            )
        )

        return best_node

    def verify_solution(self) -> bool:
        """Verify if the constructed solution satisfies all constraints."""
        positions = {node: idx for idx, node in enumerate(self.pi)}

        if set(self.pi) != set(self.graph.V):
            return False

        for v1 in self.graph.V:
            for v2 in self.graph.constraints[v1]:
                if positions[v1] > positions[v2]:
                    return False

        return True

    def greedy_construction(self) -> List[int]:
        """
        Construct a solution using a fast greedy approach.

        Returns:
            Ordered list of nodes representing the solution
        """
        self.pi = []
        in_degree = self.graph.in_degree.copy()
        candidates = deque([v for v in self.graph.V if in_degree[v] == 0])
        nodes_placed = 0

        while candidates:
            best_node = self._get_best_candidate(candidates, len(self.pi))
            if best_node is None:
                break

            self.pi.append(best_node)
            candidates.remove(best_node)
            nodes_placed += 1

            # Update in-degrees and add new candidates
            for v_next in self.graph.constraints[best_node]:
                in_degree[v_next] -= 1
                if in_degree[v_next] == 0:
                    candidates.append(v_next)

        if nodes_placed != len(self.graph.V):
            raise ValueError("Graph contains a cycle or is disconnected!")

        if not self.verify_solution():
            raise ValueError("Construction resulted in invalid solution!")

        return self.pi

In [65]:
results = []  # Store (item_name, time, cost) for each solution
for item in items_test_small:
    graph = load_instance(item)
    with Timer() as t:
        solution = DeterministicConstruction(graph)
        ordering = solution.greedy_construction()
    cost = cost_function_bit(graph, ordering)  # Cost computation outside the timing block
    results.append((item, t.delta, cost))

print(f"{'Item':<50} {'Time (s)':<10} {'Cost':<10}")
print("-" * 70)
for item, time_taken, cost in results:
    print(f"{item:<50} {time_taken:<10.6f} {cost:<10}")

Item                                               Time (s)   Cost      
----------------------------------------------------------------------
/content/test/small/inst_50_4_00006                0.000421   3184.0    
/content/test/small/inst_50_4_00007                0.000399   2694.0    
/content/test/small/inst_50_4_00002                0.000491   26152.0   
/content/test/small/inst_50_4_00009                0.000322   1471.0    
/content/test/small/inst_50_4_00005                0.000399   4178.0    
/content/test/small/inst_50_4_00008                0.000364   1542.0    
/content/test/small/inst_50_4_00003                0.000286   13128.0   
/content/test/small/inst_50_4_00004                0.000393   7328.0    
/content/test/small/inst_50_4_00001                0.000319   76753.0   
/content/test/small/inst_50_4_00010                0.000300   966.0     


## GA

In [205]:
@dataclass
class GAParameters:
    population_size: int = 100
    generations: int = 100
    elite_size: int = 10
    tournament_size: int = 5
    mutation_rate: float = 0.3
    crossover_rate: float = 0.8
    constraint_penalty: float = 100000

class GeneticAlgorithmMWCCP:
    def __init__(self, graph: 'Graph', params: GAParameters = None):
        self.graph = graph
        self.params = params or GAParameters()
        self.best_solution = None
        self.best_fitness = float('-inf')
        self.generation_stats = []

    def fitness(self, V_order: List[int]) -> float:
        """Calculate fitness with penalty"""
        crossing_cost = cost_function_bit(self.graph, V_order)

        # Penalty for violated constraints
        penalty = self._calculate_constraint_violations(V_order) * self.params.constraint_penalty
        return - crossing_cost - penalty

    def _calculate_constraint_violations(self, V_order: List[int]) -> int:
        """Count number of violated constraints"""
        violations = 0
        positions = {v: i for i, v in enumerate(V_order)}

        for v1 in V_order:
            for v2 in self.graph.constraints[v1]:
                if positions[v1] > positions[v2]:
                    violations += 1
        return violations

    def _tournament_selection(self, population: List[List[int]], fitnesses: List[float]) -> List[int]:
        """Tournament selection for parent selection"""
        tournament = random.sample(range(len(population)), self.params.tournament_size) # Select k individuals
        winner = max(tournament, key=lambda i: fitnesses[i]) # Choose winner
        return population[winner]

    def _order_crossover(self, parent1: List[int], parent2: List[int]) -> List[int]:
        """Implement Order Crossover (OX) operator"""
        size = len(parent1)
        start, end = sorted(random.sample(range(size), 2))

        # Create a mapping for fast lookup
        p1_segment = set(parent1[start:end])

        # Initialize child with the segment from parent1
        child = [-1] * size
        child[start:end] = parent1[start:end]

        # Fill remaining positions with elements from parent2
        j = end
        for i in range(size):
            current = parent2[(end + i) % size]
            if current not in p1_segment:
                child[j % size] = current
                j += 1

        return child

    def _swap_mutation(self, individual: List[int]) -> List[int]:
        """Swap mutation with variable number of swaps"""
        if random.random() < self.params.mutation_rate:
            num_swaps = random.randint(1, max(1, len(individual) // 10))
            mutated = individual.copy()
            for _ in range(num_swaps):
                i, j = random.sample(range(len(mutated)), 2)
                mutated[i], mutated[j] = mutated[j], mutated[i]
            return mutated
        return individual

    def _repair_individual(self, individual: List[int]) -> List[int]:
        in_degree = self.graph.in_degree.copy()
        adjacency_list = self.graph.constraints
        index_map = {val: idx for idx, val in enumerate(individual)}

        queue = deque(v for v in self.graph.V if in_degree[v] == 0)
        repaired = []
        remaining = set(individual)

        while queue:
            candidates = [v for v in queue if v in remaining]
            if not candidates:
                candidates = list(remaining)

            v = min(candidates, key=index_map.get)
            repaired.append(v)
            remaining.remove(v)
            queue.remove(v)

            for neighbor in adjacency_list[v]:
                if neighbor in remaining:
                    in_degree[neighbor] -= 1
                    if in_degree[neighbor] == 0:
                        queue.append(neighbor)

        return repaired

    def is_valid(self, individual: List[int]) -> bool:
        """Check if an individual satisfies all constraints."""
        positions = {v: i for i, v in enumerate(individual)}
        for v1 in individual:
            for v2 in self.graph.constraints[v1]:
                if positions[v1] > positions[v2]:
                    return False
        return True


    def run(self) -> Tuple[List[int], float]:
        """Execute the genetic algorithm"""
        # Initialize population
        population = [random.sample(self.graph.V, len(self.graph.V)) for _ in range(self.params.population_size)]
        population = [self._repair_individual(ind) if not self.is_valid(ind) else ind for ind in population]

        for generation in range(self.params.generations):
            # Evaluate fitness
            fitnesses = [self.fitness(ind) for ind in population]

            # Store statistics
            gen_best_fitness = max(fitnesses)
            gen_avg_fitness = sum(fitnesses) / len(fitnesses)
            self.generation_stats.append((gen_best_fitness, gen_avg_fitness))

            # Update best solution
            best_idx = fitnesses.index(gen_best_fitness)
            if fitnesses[best_idx] > self.best_fitness:
                self.best_fitness = fitnesses[best_idx]
                self.best_solution = population[best_idx]

            # Create next generation
            next_generation = []

            # Elitism
            elite_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i], reverse=True)
            next_generation.extend([population[i] for i in elite_indices[:self.params.elite_size]])

            # Generate offspring
            while len(next_generation) < self.params.population_size:
                if random.random() < self.params.crossover_rate:
                    parent1 = self._tournament_selection(population, fitnesses)
                    parent2 = self._tournament_selection(population, fitnesses)
                    child = self._order_crossover(parent1, parent2)
                else:
                    child = self._tournament_selection(population, fitnesses).copy()

                child = self._swap_mutation(child)
                child = self._repair_individual(child)
                next_generation.append(child)

            population = next_generation

        return self.best_solution, - self.best_fitness

    def get_statistics(self) -> List[Tuple[float, float]]:
        """Return generation statistics"""
        return self.generation_stats

def solve_mwccp(graph: 'Graph', params: GAParameters = None) -> Tuple[List[int], float]:
    """Convenience function to solve MWCCP"""
    ga = GeneticAlgorithmMWCCP(graph, params)
    solution, cost = ga.run()
    return solution, cost

In [219]:
graph = load_instance(items[0])
cost_function_bit(graph, graph.V)

23235985.0

In [223]:
params = GAParameters(
    population_size=100,
    generations=200,
    elite_size=10,
    tournament_size=10,
    mutation_rate=0.2,
    crossover_rate=0.8,
    constraint_penalty=10000000
)
# Solve the problem
solution, cost = solve_mwccp(graph, params)
print(f"Minimum weighted crossing cost: {cost}")

Minimum weighted crossing cost: 21546654.0


In [224]:
instance_name = os.path.basename(items[0])
with open(f"{instance_name}_ga.txt", "w") as f:
    f.write(instance_name + "\n")
    f.write(" ".join(map(str, solution)) + "\n")

## Ant Colony Optimization

In [50]:
#class AntColony:
#    def __init__(self, graph, alpha=1.0, beta=2.0, evaporation_rate=0.5, ant_count=10, iterations=100, top_k=3):
#        self.graph = graph
#        self.alpha = alpha
#        self.beta = beta
#        self.evaporation_rate = evaporation_rate
#        self.ant_count = ant_count
#        self.iterations = iterations
#        self.top_k = top_k
#        self.vertex_to_index = {v: i for i, v in enumerate(self.graph.V)}
#        self.index_to_vertex = {i: v for i, v in enumerate(self.graph.V)}
#
#    def initialize_pheromone_and_heuristic(self):
#        """Initialize pheromone and heuristic matrices."""
#        num_vertices = len(self.graph.V)
#        pheromone = np.ones((num_vertices, num_vertices)) / num_vertices
#
#        heuristic = np.zeros((num_vertices, num_vertices))
#        for v in self.graph.V:
#            idx = self.vertex_to_index[v]
#            heuristic[idx, :] = 1 / (1 + self.graph.in_degree[v])
#
#        return pheromone, heuristic
#
#    def construct_solution(self, pheromone, heuristic):
#        """Construct a solution probabilistically."""
#        num_vertices = len(self.graph.V)
#        unvisited = set(self.graph.V)
#        solution = []
#
#        for position in range(num_vertices):
#            idx_unvisited = [self.vertex_to_index[v] for v in unvisited]
#            probabilities = np.array([
#                (pheromone[idx][position] ** self.alpha) * (heuristic[idx][position] ** self.beta)
#                for idx in idx_unvisited
#            ])
#            probabilities /= probabilities.sum()
#
#            # Select vertex probabilistically
#            selected_idx = np.random.choice(idx_unvisited, p=probabilities)
#            vertex = self.index_to_vertex[selected_idx]
#            solution.append(vertex)
#            unvisited.remove(vertex)
#
#        if not self.is_valid(solution):
#            solution = self.repair_solution(solution)
#
#        return solution
#
#    def repair_solution(self, solution):
#        """Repair a solution to satisfy constraints."""
#        in_degree = self.graph.in_degree.copy()
#        adjacency_list = self.graph.constraints
#        index_map = {val: idx for idx, val in enumerate(solution)}
#
#        queue = deque(v for v in self.graph.V if in_degree[v] == 0)
#        repaired = []
#        remaining = set(solution)
#
#        while queue:
#            candidates = [v for v in queue if v in remaining]
#            if not candidates:
#                candidates = list(remaining)
#
#            v = min(candidates, key=index_map.get)
#            repaired.append(v)
#            remaining.remove(v)
#            queue.remove(v)
#
#            for neighbor in adjacency_list[v]:
#                if neighbor in remaining:
#                    in_degree[neighbor] -= 1
#                    if in_degree[neighbor] == 0:
#                        queue.append(neighbor)
#
#        return repaired
#
#    def is_valid(self, solution):
#        """Check if a solution satisfies all constraints."""
#        positions = {v: i for i, v in enumerate(solution)}
#        for v1 in solution:
#            for v2 in self.graph.constraints[v1]:
#                if positions[v1] > positions[v2]:
#                    return False
#        return True
#
#    def update_pheromones(self, pheromone, solutions, fitnesses):
#        """Update pheromone levels based on solutions."""
#        pheromone *= (1 - self.evaporation_rate)  # Evaporation
#
#        fitnesses = np.array(fitnesses)
#        sorted_indices = np.argsort(fitnesses)
#        for idx in sorted_indices[:self.top_k]:  # Best solutions
#            solution = solutions[idx]
#            for position, vertex in enumerate(solution):
#                v_idx = self.vertex_to_index[vertex]
#                pheromone[v_idx][position] += 1.0 / (1.0 + fitnesses[idx])  # Scale update inversely to fitness
#
#        return pheromone
#
#    def greedy_initialization(self):
#        return DeterministicConstruction(self.graph).greedy_construction()
#
#    def optimize(self):
#        """ACO algorithm for MWCCP."""
#        pheromone, heuristic = self.initialize_pheromone_and_heuristic()
#        best_solution = self.greedy_initialization()
#        best_cost = cost_function_bit(self.graph, best_solution)
#
#        for _ in range(self.iterations):
#            solutions = []
#            fitnesses = []
#
#            # Construct solutions for all ants
#            for _ in range(self.ant_count):
#                solution = self.construct_solution(pheromone, heuristic)
#                cost = cost_function_bit(self.graph, solution)
#                solutions.append(solution)
#                fitnesses.append(cost)
#
#                if cost < best_cost:
#                    best_cost = cost
#                    best_solution = solution
#
#            # Update pheromone levels
#            pheromone = self.update_pheromones(pheromone, solutions, fitnesses)
#
#        return best_solution, best_cost
#

In [44]:
class MaxMinAntSystem:
    def __init__(self, graph, alpha=1.0, beta=2.0, evaporation_rate=0.5, ant_count=10, iterations=100, tau_min=0.1, tau_max=10):
        self.graph = graph
        self.alpha = alpha  # Influence of pheromone
        self.beta = beta  # Influence of heuristic
        self.evaporation_rate = evaporation_rate
        self.ant_count = ant_count
        self.iterations = iterations
        self.tau_min = tau_min
        self.tau_max = tau_max

        # Map vertex IDs to indices for consistent matrix access
        self.vertex_to_index = {v: i for i, v in enumerate(self.graph.V)}
        self.index_to_vertex = {i: v for i, v in enumerate(self.graph.V)}

    def initialize_pheromone_and_heuristic(self):
        """Initialize pheromone and heuristic matrices."""
        num_vertices = len(self.graph.V)
        pheromone = np.ones((num_vertices, num_vertices), dtype=np.float64) * self.tau_max

        heuristic = np.zeros((num_vertices, num_vertices), dtype=np.float64)
        for v in self.graph.V:
            idx = self.vertex_to_index[v]
            # Use a heuristic metric
            heuristic[idx, :] = 1 / (1 + self.graph.in_degree[v])
            #neighbors = self.graph.node_edges[v]
            #avg_w = np.mean([neigh[1] for neigh in neighbors])
            #heuristic[idx, :] = 1 / (1 + avg_w)

        return pheromone, heuristic


    def construct_solution(self, pheromone, heuristic):
        """Construct a solution probabilistically."""
        num_vertices = len(self.graph.V)
        unvisited = set(self.graph.V)
        solution = []

        for position in range(num_vertices):
            probabilities = []
            for v in unvisited:
                idx = self.vertex_to_index[v]
                probabilities.append(
                    (pheromone[idx][position] ** self.alpha) * (heuristic[idx][position] ** self.beta)
                )
            probabilities = np.array(probabilities) / sum(probabilities)

            # Select vertex probabilistically
            vertex = random.choices(list(unvisited), weights=probabilities, k=1)[0]
            solution.append(vertex)
            unvisited.remove(vertex)

        if not self.is_valid(solution):
            solution = self.repair_solution(solution)

        return solution

    def repair_solution(self, solution):
        """Repair a solution to satisfy constraints."""
        in_degree = self.graph.in_degree.copy()
        adjacency_list = self.graph.constraints
        index_map = {val: idx for idx, val in enumerate(solution)}

        queue = deque(v for v in self.graph.V if in_degree[v] == 0)
        repaired = []
        remaining = set(solution)

        while queue:
            candidates = [v for v in queue if v in remaining]
            if not candidates:
                candidates = list(remaining)

            v = min(candidates, key=index_map.get)
            repaired.append(v)
            remaining.remove(v)
            queue.remove(v)

            for neighbor in adjacency_list[v]:
                if neighbor in remaining:
                    in_degree[neighbor] -= 1
                    if in_degree[neighbor] == 0:
                        queue.append(neighbor)

        return repaired

    def is_valid(self, solution):
        """Check if a solution satisfies all constraints."""
        positions = {v: i for i, v in enumerate(solution)}
        for v1 in solution:
            for v2 in self.graph.constraints[v1]:
                if positions[v1] > positions[v2]:
                    return False
        return True

    def update_pheromones(self, pheromone, best_solution, best_cost):
        """Update pheromone levels based on the best solution."""
        pheromone *= (1 - self.evaporation_rate)  # Evaporation

        for position, vertex in enumerate(best_solution):
            idx = self.vertex_to_index[vertex]
            pheromone[idx][position] += 1.0 / (1.0 + best_cost)

        # Apply pheromone bounds
        pheromone = np.clip(pheromone, self.tau_min, self.tau_max)

        return pheromone

    def ant_colony_optimization(self):
        """MMAS algorithm for MWCCP"""
        pheromone, heuristic = self.initialize_pheromone_and_heuristic()

        best_solution = None
        best_cost = float('inf')

        for _ in range(self.iterations):
            solutions = []
            fitnesses = []

            for _ in range(self.ant_count):
                solution = self.construct_solution(pheromone, heuristic)
                cost = cost_function_bit(self.graph, solution)
                solutions.append(solution)
                fitnesses.append(cost)

                if cost < best_cost:
                    best_cost = cost
                    best_solution = solution

            # Update pheromone with only the global best solution
            pheromone = self.update_pheromones(pheromone, best_solution, best_cost)

            # Dynamically adjust tau_min and tau_max
            self.tau_max = 1.0 / (1.0 - self.evaporation_rate) * (1.0 / best_cost)
            self.tau_min = self.tau_max / (2 * len(self.graph.V))

        return best_solution, best_cost

In [48]:
graph = load_instance(items_test_small[0])
cost_function_bit(graph, graph.V)

82627.0

In [49]:
graph = load_instance(items_test_small[0])

aco = MaxMinAntSystem(graph, alpha=1.6, beta=1.4, evaporation_rate=0.3, ant_count=20, iterations=100)
solution, cost = aco.ant_colony_optimization()
print(f"Best solution: {solution}")
print(f"Minimum weighted crossing cost: {cost}")

Best solution: [30, 44, 41, 38, 26, 28, 32, 40, 42, 49, 48, 31, 36, 37, 27, 46, 34, 35, 50, 45, 29, 39, 43, 47, 33]
Minimum weighted crossing cost: 74701.0


In [21]:
instance_name = os.path.basename(items[1])
with open(f"{instance_name}_dt.txt", "w") as f:
    f.write(instance_name + "\n")
    f.write(" ".join(map(str, solution)) + "\n")

IndexError: list index out of range